<a href="https://colab.research.google.com/github/abellokeefe/DS-Unit-1-Sprint-2-Statistics/blob/master/Copy_of_Anna_Bell_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

# Import / Load

In [0]:
#import
import pandas as pd
import numpy as np
from scipy import stats

In [0]:
# load/label/format data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

column_headers = ['party','handicapped-infants','water-project',
                  'budget','physician-fee-freeze', 'el-salvador-aid',
                  'religious-groups','anti-satellite-ban',
                  'aid-to-contras','mx-missile','immigration',
                  'synfuels', 'education', 'right-to-sue','crime','duty-free',
                  'south-africa']

df = pd.read_csv('house-votes-84.data', names=column_headers)
df = df.replace({'y': 1, 'n': 0, '?': np.NaN})

print(df.shape)
df.head()

--2020-04-14 01:19:59--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2020-04-14 01:20:00 (495 KB/s) - ‘house-votes-84.data’ saved [18171/18171]

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


# Clean up data

In [0]:
#make dataframe of democratic votes only
dems = df[df['party']=='democrat']
dems.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
#make dataframe of all democratic votes per bill
dems_sum = pd.DataFrame(dems.sum(axis=0, skipna=True)).drop('party').rename(columns={0:'votes'}) 
dems_sum[:5] #not to be confused with dim-sum

,votes
handicapped-infants,156
water-project,120
budget,231
physician-fee-freeze,14
el-salvador-aid,55


In [0]:
#make dataframe of republican votes only
reps = df[df['party']=='republican']
reps.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
#make dataframe of all republican votes per bill
reps_sum = pd.DataFrame(reps.sum(axis=0, skipna=True)).drop('party').rename(columns={0:'votes'})
reps_sum[:5]

,votes
handicapped-infants,31
water-project,75
budget,22
physician-fee-freeze,163
el-salvador-aid,157


In [0]:
# make dataframe that displays the sum of both
dems_reps_sum = pd.DataFrame()
dems_reps_sum['dem votes'] = dems_sum['votes']
dems_reps_sum['rep votes'] = reps_sum['votes']
dems_reps_sum

,dem votes,rep votes
handicapped-infants,156,31
water-project,120,75
budget,231,22
physician-fee-freeze,14,163
el-salvador-aid,55,157
religious-groups,123,149
anti-satellite-ban,200,39
aid-to-contras,218,24
mx-missile,188,19
immigration,124,92


# T-test from scratch

# Run T-Tests

In [0]:
# 1)
# Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

# H_0: votes of dems and reps for "budget" are the same (LOL)
# alt_hyp: votes of dems and reps for "budget" are significantly different

t_test = stats.ttest_ind(dems['budget'], reps['budget'], nan_policy='omit')
t_stat = t_test[0]
p_val = t_test[1]
print("Test Statistic:", t_stat)
print("P-Value:", p_val)

if p_val <0.05:
  print("we reject the null hypothesis")

else:
    print("we accept null hypothesis")

result_1 = t_test

Test Statistic: 23.21277691701378
P-Value: 2.0703402795404463e-77
we reject the null hypothesis


In [0]:
# let's just fact check our results realquick
dems_budget_sum = dems_reps_sum.iloc[2,0]
reps_budget_sum = dems_reps_sum.iloc[2,1]

party_difference = dems_budget_sum - reps_budget_sum
percent_diff = party_difference / ((dems_budget_sum+reps_budget_sum)/2)*100

print("Democrats voted" , int(party_difference) , "more times than Republicans on the budget")
print("That's a", int(percent_diff), "% difference.")
print("\nConfirmed: Democrats love money.")

Democrats voted 209 more times than Republicans on the budget
That's a 165 % difference.

Confirmed: Democrats love money.


In [0]:
# 2) 
# Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

# H_0: votes of dems and reps for "education" are the sam=
# alt_hyp: votes of dems and reps for "education" are significantly different

t_test = stats.ttest_ind(dems['education'], reps['education'], nan_policy='omit')
t_stat = t_test[0]
p_val = t_test[1]

print("Test Statistic:", t_stat)
print("P-Value:", p_val)

if p_val <0.05:
  print("we reject the null hypothesis")

else:
    print("we accept null hypothesis")

result_2 = t_test

Test Statistic: -20.500685724563073
P-Value: 1.8834203990450192e-64
we reject the null hypothesis


In [0]:
#fact check
dems_ed_sum = dems_reps_sum.iloc[11,0]
reps_ed_sum = dems_reps_sum.iloc[11,1]

party_difference = reps_ed_sum - dems_ed_sum
percent_diff = party_difference / ((dems_ed_sum+reps_ed_sum)/2)*100

print("Republicans voted voted" , int(party_difference) , "more times than Democrats on the education bill")
print("That's a", int(percent_diff), "% difference.")
print("\nConfirmed: Republicans love giving money to education.")

Republicans voted voted 99 more times than Democrats on the education bill
That's a 115 % difference.

Confirmed: Republicans love giving money to education.


In [0]:
# Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 

# H_0: votes of dems and reps for "religious-groups" are the same 
# alt_hyp: votes of dems and reps for "religious-groups" are significantly different
t_test = stats.ttest_ind(dems['religious-groups'], reps['religious-groups'], nan_policy='omit')
t_stat = t_test[0]
p_val = t_test[1]

print("Test Statistic:", t_stat)
print("P-Value:", p_val)

if p_val <0.05:
  print("we reject the null hypothesis")

else:
    print("we accept null hypothesis")

result_3 = t_test

Test Statistic: -9.737575825219457
P-Value: 2.3936722520597287e-20
we reject the null hypothesis


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

In [0]:
#Stretch Goal 1:
def t_test(samp_A, samp_B):
  t_stat, p_val = stats.ttest_ind(samp_A, samp_B, nan_policy='omit')

  if p_val <0.05:
    return print("we reject the null hypothesis")

  else:
    return print("we accept null hypothesis")

In [0]:
#Stretch Goal 2:

#as a function

def under_the_hood(samp_A, samp_B, a): #a = alpha, is the significance value (if p_value < alpha (0,05) then we reject H_0)
  #get means
  mean_A = np.mean(samp_A)                  #mean of sample A
  mean_B = np.mean(samp_B)                  #mean of sample B

  #get stdevs
  stdv_A = np.std(samp_A)                   #standard deviation of sample A
  stdv_B = np.std(samp_B)                   #standard deviation of B

  #get sample size for each
  N_A = len(samp_A) - samp_A.isnull().sum() #sample size A
  N_B = len(samp_B) - samp_B.isnull().sum() #sample size B

  #get total number of observations
  N = N_A + N_B

  #Degrees of Freedom
  degrees_freedom = N-1

  #get standard errors for each 
  st_err_A = stdv_A/np.sqrt(N_A)            #standard error of sample A
  st_err_B = stdv_B/np.sqrt(N_B)            #standard error of sample B

  #get standard error of difference between both samples
  std_err_diff = np.sqrt(st_err_A**2+st_err_B**2)

  #get t statistic
  t_stat=(mean_A - mean_B) / std_err_diff

  #get p-value
  p = stats.t.cdf(1 - (t_stat),df=degrees_freedom)*2 #multiplied by 2 to get both tails
  #return all the things
  return t_stat, degrees_freedom, p

In [0]:
#Comparing Results: 

#scipy
print("scipy function results for t statistic: " + str(result_1[0]))
print("scipy funciton results for p value: " + str(result_1[1]))

#under_the_hood results
t_stat, deg_free, p_val = under_the_hood(dems['budget'], reps['budget'], 0.05)
print("\nfunction result for t statistic: ", t_stat)
print("function result for p_value: ", p_val)


if p_val < 0.05:
  print("we reject the null hypothesis\n")

else:
    print("we accept null hypothesis\n")

scipy function results for t statistic: 23.21277691701378
scipy funciton results for p value: 2.0703402795404463e-77

function result for t statistic:  22.854016074618663
function result for p_value:  2.0560371114417406e-71
we reject the null hypothesis



References:

* T Score Formula: Calculate in Easy Steps. Retrieved From: https://www.statisticshowto.com/probability-and-statistics/t-distribution/t-score-formula/

* Student's T Test: Definition and Examples. Retrieved from: https://www.statisticshowto.com/probability-and-statistics/t-test/

* Calculating the P-value in a t test for a mean. Khan Academy. Retrieved from: https://www.khanacademy.org/math/ap-statistics/tests-significance-ap/one-sample-t-test-mean/e/p-value-t-test-mean

* T-test using Python and Numpy (this one has sample code). Towards Data Science. Retrieved from: https://www.khanacademy.org/math/ap-statistics/tests-significance-ap/one-sample-t-test-mean/e/p-value-t-test-mean

* SciPy.org ttest_ind. Retrieved from: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

* SciPy sourcecode. Github. Retrieved from: https://github.com/scipy/scipy/blob/v1.4.1/scipy/stats/stats.py#L5141-L5266
